<a href="https://colab.research.google.com/github/AzucenaMV/top2000-dashboard/blob/main/top2000_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzymatcher

     |████████████████████████████████| 50 kB 6.8 MB/s 
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13918 sha256=a0808b47c21a77651d2e12af65586ee3d9291921b2f65c96655dc6443f1d26f5
  Stored in directory: /root/.cache/pip/wheels/1d/a8/cb/6f8902aa5457bd71344e00665c230e9c45255b3f57f2194a0f
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149860 sha256=b8ea6dd0708d90a8e590f6d173a0d40efcd296e01f78c762c0b7f7377ca37e7b
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built metaphone python-Levenshtein


In [ ]:
import pandas as pd
import requests
import os
import fuzzymatcher

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load data

path = 'drive/MyDrive/JADS/DataVizProject/Code/'
file_gender = "Gender_Dataset_1.3.csv"
file_top = "top2000.csv"

df = pd.read_csv(os.path.join(path,file_top))
dfgender = pd.read_csv(os.path.join(path,file_gender))
dfgender.head()

,chartmetric_id,name,artist_country,pronoun,gender,is_band,genre
0,1.0,Various Artists,GB,they/them,mixed,True,"{scorecore,soundtrack,Pop,Classical,""Christian..."
1,2.0,Massive Attack,GB,they/them,NaN,True,"{""trip hop"",Dance,downtempo,Electronic,""big be..."
2,3.0,Squirrel Nut Zippers,US,they/them,NaN,True,"{Jazz,swing,Rock,Pop,""north carolina roots""}"
3,4.0,Bob Dylan,US,he/him,male,False,"{""roots rock"",folk,singer-songwriter,""classic ..."
4,5.0,10cc,GB,they/them,NaN,True,"{Pop,""progressive rock"",singer-songwriter,""fol..."


In [ ]:
dfgender.rename(
    columns = {"name" : "artist"},
    inplace = True
)

In [ ]:
def clean_name(df, new_col = 'artist_clean', old_col = 'artist'):
  df.dropna(subset = [old_col], inplace = True)
  df[new_col] = df[old_col].str.lower()
  df[new_col] = df[new_col].str.lstrip()
  df[new_col] = df[new_col].str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')
  return df

In [ ]:
df = clean_name(df)
dfgender = clean_name(dfgender)

In [ ]:
df.head()

,artist,song,released_date,highest_position,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,artist_clean
0,Queen,Bohemian Rhapsody,1975,1,1,1,1,1,1,1,2,1,1,1,1,2,1,1,1,2,2,1,1,1,1,2,1,queen
1,Danny Vera,Roller Coaster,2019,1,×,×,×,×,×,×,×,×,×,×,×,×,×,×,×,×,×,×,×,×,4,1,2,danny vera
2,Procol Harum,A Whiter Shade of Pale,1967,3,10,18,23,20,22,28,22,25,47,26,50,42,65,61,83,98,105,119,117,153,162,153,3,procol harum
3,Eagles,Hotel California,1977,1,2,4,3,3,2,2,3,3,3,2,2,1,2,2,2,1,3,2,2,2,2,3,4,eagles
4,Billy Joel,Piano Man,1974,3,121,83,57,58,60,66,49,54,34,50,45,45,29,36,18,7,6,4,4,3,3,4,5,billy joel


In [ ]:
# removing duplicates
dfgender.drop_duplicates(keep = 'first', inplace = True, subset = ['artist_clean'])

# merging dfs
dfgender_match = dfgender[dfgender.artist_clean.isin(df.artist_clean)]
df_merged = df.merge(dfgender_match, how = 'left', on = ['artist_clean'], suffixes = ('','_right'))

In [ ]:
print(df_merged.chartmetric_id.isna().sum())
print(df_merged.chartmetric_id.notna().sum())

762
3875


In [ ]:
df_merged.tail(5)

,artist,song,released_date,highest_position,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,artist_clean,chartmetric_id,artist_right,artist_country,pronoun,gender,is_band,genre
4632,Zucchero Fornaciari,Diamante,1990,725,—,—,—,—,—,—,1250,967,725,1319,1171,1002,1053,1268,1433,1393,1466,—,1975,—,—,—,—,zucchero fornaciari,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4633,Zucchero Fornaciari & Paul Young,Senza una donna (Without a Woman),1991,523,605,—,788,1163,811,777,741,706,523,665,923,763,832,1288,1372,1212,1189,1273,1558,—,—,—,—,zucchero fornaciari & paul young,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4634,ZZ en de Maskers,Ik heb genoeg van jou,1965,1102,1241,—,1393,1546,1242,1231,1102,1432,1205,1276,—,—,—,—,—,—,—,—,—,—,—,—,—,zz en de maskers,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4635,ZZ en de Maskers,La comparsa,1964,525,—,631,708,854,525,572,547,578,529,561,—,1146,1339,1666,1757,1340,1437,—,—,—,—,—,—,zz en de maskers,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4636,ZZ Top,Legs,1985,1412,—,—,—,—,—,—,—,—,—,—,1416,—,1412,1474,1587,1412,1733,1920,—,—,—,—,—,zz top,489.0,ZZ Top,US,they/them,male,True,"{rock,""album rock"",blues-rock,""classic rock"",""..."


In [ ]:
df_merged['index'] = df_merged.index

In [ ]:
# subset of not founded
df_incomplete = df_merged[df_merged.chartmetric_id.isna()]
df_incomplete = df_incomplete[list(df.columns[0:(df.columns.get_loc("artist_clean")+1)]) + ['index']]

# subset of founded
df_complete = df_merged[df_merged.chartmetric_id.notna()]

In [ ]:
# fuzzy match in not founded
df_fuzzy = fuzzymatcher.fuzzy_left_join(df_incomplete, dfgender, left_on = "artist_clean", right_on = "artist_clean")

In [ ]:
df_fuzzy[['artist_left','artist_right']]

,artist_left,artist_right
0,Prince & The Revolution,Ty the Prince
42,Lady Gaga & Bradley Cooper,Lady Gaga
92,Klein Orkest,Klein
142,Wim Sonneveld,Wim
155,Ramses Shaffy & Liesbeth List,Liesbeth List
...,...,...
23701,Zucchero Fornaciari,Zucchero
23702,Zucchero Fornaciari,Zucchero
23703,Zucchero Fornaciari & Paul Young,Paul Young
23705,ZZ en de Maskers,"Zz, The Slept On"


In [ ]:
df_complete.columns

Index(['artist', 'song', 'released_date', 'highest_position', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', 'artist_clean', 'chartmetric_id',
       'artist_right', 'artist_country', 'pronoun', 'gender', 'is_band',
       'genre', 'index'],
      dtype='object')

In [ ]:
df_fuzzy.columns

Index(['best_match_score', '__id_left', '__id_right', 'artist_left', 'song',
       'released_date', 'highest_position', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', 'artist_clean_left', 'index', 'chartmetric_id', 'artist_right',
       'artist_country', 'pronoun', 'gender', 'is_band', 'genre',
       'artist_clean_right'],
      dtype='object')

In [ ]:
# Concat both dfs
df_fuzzy.columns = df_fuzzy.columns.str.replace('_left','')
df_fuzzy = df_fuzzy[df_complete.columns]
df_final = pd.concat([df_fuzzy,df_complete])

In [ ]:
df_final = df_final.sort_values(by = 'index', ascending = True)
df_final.to_csv(os.path.join(path,'top2000_gender.csv'), index = False)